In [194]:
from PIL import Image, ImageChops
import numpy as np
from time import time_ns
import math

In [195]:
def printAudit(test_name, delta_time, audit):
  print(f'\nTest {test_name} passed: {(delta_time) / math.pow(10, 9)}s')
  for i in audit:
    name = i.get('name')
    input_size = i.get('input_size')
    output_size = i.get('output_size')
    print(f'Image {name}; input_size: {input_size}; output_size: {output_size}')

In [196]:
'''def getImages():
    image_1 = Image.open('input/first.png', 'r').convert('RGB')
    image_2 = Image.open('input/second.png', 'r').convert('RGB')
    image_3 = Image.open('input/third.png', 'r').convert('RGB')
    image_4 = Image.open('input/fourth.png', 'r').convert('RGB')
    image_5 = Image.open('input/fifth.png', 'r').convert('RGB')
    image_6 = Image.open('input/sixth.png', 'r').convert('RGB')
    image_7 = Image.open('input/seventh.png', 'r').convert('RGB')
    image_8 = Image.open('input/eigth.png', 'r').convert('RGB')
    image_9 = Image.open('input/ninth.png', 'r').convert('RGB')
    return {'first': image_1, 'second': image_2, 'third': image_3, 'fourth': image_4, 'fifth': image_5, 'sixth': image_6, 'seventh': image_7, 'eigth': image_8, 'ninth': image_9}'''

In [197]:
def calculate_template_space(temp_side_length): #длина структурного элемента
  return int(temp_side_length/2)   #чтобы найти отступ от края, т.е операции не будут применяться на краях

In [200]:
def dilation(image, template_side_length, template):
    new_image = np.zeros(image.shape, image.dtype)
    template_space = calculate_template_space(template_side_length) #вычисляем отступ от краев
    half_template = int((template_side_length - 1) / 2) #для центрирования относительно текущего пикселя

    #Проверка на вшивость
    is_binary = np.isin(image, [0, 1]).all()

    for y in range(template_space, new_image.shape[0] - template_space):
        for x in range(template_space, new_image.shape[1] - template_space):  #чтоб не выехал без загранника из страны
            if is_binary:
                has_one = False
                for c in range(-half_template, half_template + 1):  #Если в окне операции находится хотя бы одна единица,то в центральную позицию окна заносится 1.
                    for d in range(-half_template, half_template + 1):
                        if template[d + half_template, c + half_template] == 1 and image[y + d, x + c] == 1:
                            has_one = True
                            break
                    if has_one:
                        break
                new_image[y, x] = 1 if has_one else 0
            else:
                #для полутона
                maximum = -1  
                for c in range(-half_template, half_template + 1):   #в окне операции отыскивается элемент с максимальным значением и помещается в центральную позицию.
                    for d in range(-half_template, half_template + 1):
                        if template[d + half_template, c + half_template] == 1:
                            val = image[y + d, x + c]
                            maximum = max(maximum, val)
                new_image[y, x] = maximum

    return new_image

In [205]:
import os

def dilation_test(iterations: int, template_n: int):
    def getImages():
        base_path = 'input/'
        return {
            'first': base_path + 'first.png',
            'second': base_path + 'second.png',
            'third': base_path + 'third.png',
            'fourth': base_path + 'fourth.png',
            'fifth': base_path + 'fifth.png',
            'sixth': base_path + 'sixth.png',
            'seventh': base_path + 'seventh.png',
            'eigth': base_path + 'eigth.png',
            'ninth': base_path + 'ninth.png',
        }
    images = getImages()
    start_time = time_ns()
    audit = []
    template = np.ones((template_n, template_n), dtype=np.uint8) #структурирующий элемент 

    for image_name, image_path in images.items():
        image = Image.open(image_path).convert('L')
        original_image_array = np.array(image, dtype=np.uint8)
        is_binary = np.isin(original_image_array, [0, 255]).all()

        processed_image_array = original_image_array.copy()
        for _ in range(iterations):
            processed_image_array = dilation(processed_image_array, template_n, template)

        # Вычисление разностного изображения в зависимости от типа
        if is_binary:
            diff_image_array = np.bitwise_xor(original_image_array // 255, processed_image_array // 255) * 255
        else:
            diff_image_array = np.abs(original_image_array - processed_image_array)

        processed_image = Image.fromarray(processed_image_array)
        diff_image = Image.fromarray(diff_image_array.astype(np.uint8))

        output_path = f'output/{image_name}_processed.png'
        diff_path = f'output/{image_name}_diff.png'
        processed_image.save(output_path)
        diff_image.save(diff_path)

        audit.append({'name': image_name, 'output_path': output_path, 'diff_path': diff_path})

    end_time = time_ns()
    for record in audit:
        print(f"Image: {record['name']}, Processed: {record['output_path']}, Diff: {record['diff_path']}")

    print(f"Total processing time: {(end_time - start_time) / 1e9} seconds")

In [203]:
def test():
    dilation_test(1, 3)

In [204]:
if __name__ == '__main__':
  test()

Image: first, Processed: output/first_processed.png, Diff: output/first_diff.png
Image: second, Processed: output/second_processed.png, Diff: output/second_diff.png
Image: third, Processed: output/third_processed.png, Diff: output/third_diff.png
Image: fourth, Processed: output/fourth_processed.png, Diff: output/fourth_diff.png
Image: fifth, Processed: output/fifth_processed.png, Diff: output/fifth_diff.png
Image: sixth, Processed: output/sixth_processed.png, Diff: output/sixth_diff.png
Image: seventh, Processed: output/seventh_processed.png, Diff: output/seventh_diff.png
Image: eigth, Processed: output/eigth_processed.png, Diff: output/eigth_diff.png
Image: ninth, Processed: output/ninth_processed.png, Diff: output/ninth_diff.png
Total processing time: 1127.6635205 seconds
